# データアナリティクスII（山本担当モジュール）レポート課題
関係データベースを利用して購買データを管理している仮想の小売店（杏森堂）から，データ分析の依頼を受けたとする．下記「SQL利用環境セットアップ」を実行することで入手できるデータセット（& 分析環境）を利用して，下記4つの課題に対する回答を作成しなさい．

---

## SQL利用環境セットアップ
SQLiteはWindows/Mac/LinuxどのOSでもインストールして利用することができる．本演習は環境構築の手間をできるだけ減らすために，Google Colaboratory上でSQLiteを用いる．まず，以下のコードが書かれた箇所（セル）を実行しよう．すると，Google Colaboratory上でSQLiteを実行するための環境が用意される．

なお，Google Colaboratory上でコードを実行するには，以下の2つの方法がある：
* 「再生ボタン」をクリックする
* コードが書かれたセルをクリックし，Shift + Enterキーを押す

In [ ]:
# Google Colaboratory上のSQLiteをアップグレードする
import sys
if 'google.colab' in sys.modules:
    !gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi -O /usr/lib/python3.7/lib-dynload/

上記コードを実行したら，アップグレード版SQLiteを使うために，Google Colaboratoryを再起動させよう．
再起動の方法は，画面上部の`Runtime`から`Restart Runtime`をクリックする．

再起動が完了したら，コードを実行しよう．以下のコードはGoogle Colab（Jupyter）上でSQLを対話的に利用するためのおまじないである．

In [ ]:
# ipython-sqlがインストールされていない場合
!pip install ipython-sql

英文が数行表示された後，SQLiteがインストールされる． 

続いて，演習で用いる購買データが収められたSQLiteデータベースをダウンロードしよう． 以下のコードを実行すると，Google Colaboratory上にデータベースがダウンロードされる．

In [ ]:
# SQLiteデータベース（data-analytics-lecture.db）のファイルをダウンロード
import os
if not os.path.exists('data/data-analytics-lecture.db'):
    !wget -P data https://github.com/hontolab-courses/data-analytics-2022/raw/main/data/data-analytics-lecture.db

データベースがダウンロードできたら，データをロードしておこう．
以下のコードを実行すると，Google Colaboratory上でダウンロードしたSQLiteデータベースを操作できるようになる．

In [ ]:
%load_ext sql
%sql sqlite:///data/data-analytics-lecture.db

なお，Google Colaboratoryはセッションを閉じると，環境が初期化され，以前に自前でインストールしたライブラリが削除される．
Google Colaboratoryを使って演習に取り組む際には，上記コードを毎回忘れず実行しよう．

---

## Google Colaboratory上でのSQLの実行方法

Google Colaboratoryでは，以下のようにセルの冒頭に`%%sql`と書き，2行目以降にSQL文を書くとSQLを実行することができる．
試しに以下のSQLを実行してみよう．

In [ ]:
%%sql

SELECT
    name AS table_name
FROM
    sqlite_master
WHERE
    type = 'table';

上記SQLは，データベースに格納されたテーブルの一覧を表示するものである．
ダウンロードしたSQLiteデータベース，つまり今回扱う杏森堂のデータベースには，**6つ**のテーブルが格納されていることが分かる．なお，対象データベースの構造が記された実体関連図（ER図）は[コチラ](https://github.com/hontolab-courses/data-analytics-2022/raw/main/doc/ER-diagram.pdf)から確認することできる．

課題とドリルに取り組む前に，あと1つ，別のSQLを実行してみよう．
以下のSQLを実行すると，`receipt`テーブルの最初の10件を表示することができる
（SQL文に`LIMIT`を付けないと，テーブル内の全データが表示されてしまうので注意）．

In [ ]:
%%sql

SELECT * FROM receipt LIMIT 10; 

## ★ SQLの実行結果をCSVに保存したい場合
以下のように，セルの冒頭に`%%sql 一時保存先の変数名 <<`と書き，2行目以降にSQL文を書いて実行しよう．

In [ ]:
%%sql records <<

SELECT * FROM receipt LIMIT 10; 

その後以下のコマンドを実行することで，SQLの実行結果を格納したCSVファイルを作成することができる．作成したCSVファイルは，画面左のサイドバーにあるファイル一覧からダウンロードできる．

In [ ]:
# records.csvは任意のファイル名にできる
records.csv(filename="data/records.csv")

---

## 課題内容

### 課題1（配点30％）
杏森堂から提供された購買履歴が格納されたデータベースを用いて，これまでに何名の顧客が何回（何日）ショッピングを行ったのかを分析したい．

レシート明細テーブル（`receipt`）を用いて顧客の購買頻度を分析し，以下の項目について分析結果を表示するSQL文を書き，実行結果とともに示しなさい：
* 購買頻度（これまでに店舗を利用した日数）
* 購買頻度に対応する顧客の数
* 該当する購買頻度以下の顧客数の累積値

ただし，顧客ID（`customer_id`）が"Z"から始まるのものは非会員を表すため，除外して分析すること．

※ 以下は，SQL実行結果のイメージである：

| 購買頻度（日数） | 購買頻度に対応する顧客の数 | 顧客数累積値 |
| ---- | ---- | ---- |
| 1 | 2761 | 2761 |
| 2 | 1499 | 4260 |
| 3 | 881 | 5141 |
| ... | ... | ... |

In [ ]:
%%sql


### 課題2（配点30％）
課題2-1，課題2-2のいずれかを選択し，結果を得るためのSQL文を書き，実行結果とともに示しなさい．

#### 課題2-1: 月別売上の対昨年比
レシート明細テーブル（`receipt`）には2017年1月から2019年10月の間の購買情報が記録されている．2017年から2019年までの期間の売上を把握するために，1ヶ月ごとに以下の情報を集約表示せよ：
* 年月（`year_month`）
* 購買回数（`purchase_freq`）
* 月間売上高（`total_amount`）
* 購買1回あたりの平均購入額（`avg_amount_per_purchase`）
* 当該月の前年売上高（`total_amount_last_year`）
* 売上の対前年比（`ratio`）

※ 以下は，SQL実行結果のイメージである：

| year_month | purchase_freq | total_amount | avg_amount_per_purchase | total_amount_last_year | ratio |
| ---- | ---- | ---- | ---- | ---- | ---- |
| 2017-01 | 1405 | 902056 | 642 | NULL | NULL  |
| 2017-02 | 1219 | 764413 | 627 | NULL | NULL  |
| ... | ... | ... | ... | ... | ... |
| 2017-12 | 1467 | 939654 | 640 | NULL | NULL  |
| 2018-01 | 1473 | 944509 | 641 | 902056 | 1.05 |
| 2018-02 | 1387 | 864128 | 623 | 764413 | 1.13 |
| ... | ... | ... | ... | ... | ... |
| 2019-10 | 1750 | 1143062 | 653 | 1069939 | 1.07 |

In [ ]:
%%sql


#### 課題2-2: ABC分析
ABC分析は販売戦略を考えるために，売上によって商品をランク付けする手法である．
一般に，ABC分析では売上総額の
* 上位0〜70％の商品をAランク
* 上位70〜90％の商品をBランク
* 上位90〜100%の商品をCランク

とするランク付けを行う．

商品カテゴリ「菓子（`category_major_cd = 08`）」に属する商品について，小区分（`category_small_name`）ごとに売上を集計し，菓子カテゴリの売上総額に占める割合（構成比）を計算せよ．
また，売上がN位の小区分の行には売上額上位N位までの構成比累計，および構成比累計に基づくABC分析のランク付け結果も表示せよ．

※ 以下は，SQL実行結果のイメージである：

| カテゴリ名 | 売上 | 構成比 | 累積構成比 | ランク |
| ---- | ---- | ---- | ---- | ---- |
| 煎餅 | 194211 | 0.1306 | 0.1306 | A |
| 菓子詰め合わせ | 129580 | 0.0871 | 0.2178 | A |
| ナッツ類 | 129020 | 0.0867 | 0.3046 | A |
| ... | ... | ... | ... | ... |
| キャンディー | 46177 | 0.0310 | 0.7208 | B |
| ... | ... | ... | ... | ... |
| その他珍味 | 24086 | 0.0162 | 0.9073 | C |
| ... | ... | ... | ... | ... |

In [ ]:
%%sql


### 課題3（配点30％）
課題3-1から3-4の4題から1題を選択し回答しなさい．なお，課題3については，2題以上解いた学生には1題につき最大10％を加点する．

※ Google Colabで実行したSQLの結果をCSVファイルとして保存するには，[こちらのウェブページ](https://nbviewer.org/github/hontolab-courses/data-analytics-2022/blob/main/notebook/day-04.ipynb)の「★SQLの実行結果をCSVに保存したい場合」を参考にすること．

#### 課題3-1
顧客ごとに購買頻度（購買回数）を求め，購買頻度の分布を説明するためのグラフを作成せよ．また，グラフから読み取れることを文章で記述せよ．ただし，顧客ID（`customer_id`）が"Z"から始まるのものは非会員を表すため，除外して分析すること．

なおレポートには，顧客ごとに購買頻度（購買回数）を求めるためのSQL文も付しておくこと（SQL文の実行結果は掲載する必要なし）．

In [ ]:
%%sql


#### 課題3-2
都道府県別に年月別の売上総額を求め，その売上総額の変化を比較するためのグラフを作成せよ．また，グラフから読み取れることを文章で記述せよ．

なおレポートには，都道府県別の年月別売上総額を求めるためのSQL文も付しておくこと（SQL文の実行結果は掲載する必要なし）．

In [ ]:
%%sql


#### 課題3-3
顧客一人あたりの売上総額の分布を，千葉県の店舗ごとに比較するためのグラフを作成せよ．また，グラフから読み取れることを文章で記述せよ．ただし，顧客ID（`customer_id`）が"Z"から始まるのものは非会員を表すため，除外して分析すること．

なおレポートには，店舗ごとにその店舗を利用した各顧客の売上総額を集計するSQL文も付しておくこと（SQL文の実行結果は掲載する必要なし）．

In [ ]:
%%sql


#### 課題3-4
千葉県にある店舗間で顧客が購入する菓子の傾向が異なるかどうかを分析したい．千葉県にある店舗と菓子の中カテゴリ（cateogory_medium_cd）ごとに菓子の購入量（個数）を集計し，購入された菓子の中カテゴリの内訳を店舗間で比較するためのグラフを作成せよ．また，グラフから読み取れることを文章で記述せよ．

なおレポートには，購入量を集計するためのSQL文も付しておくこと（SQL文の実行結果は掲載する必要なし）．

In [ ]:
%%sql


### 課題4（配点10％）
データアナリティクスIIの山本担当パートについて，良かった点および改善すべき点を記しなさい．